In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
from azure.identity import DefaultAzureCredential


subscriptionId = "9d295860-44e3-44bb-ade9-235cc45c68ba" #replace with your subscription ID

token_credential = DefaultAzureCredential()
token = token_credential.get_token('https://management.azure.com/.default')
headers = {'Authorization': 'Bearer ' + token.token}

unique_region_names = set()

locations_request = f"https://management.azure.com/subscriptions/{subscriptionId}/locations?api-version=2021-04-01"

response = requests.get(locations_request, headers=headers)

data = json.loads(response.text)

for item in data["value"]:
    unique_region_names.add(item["name"])

regions_list = list(unique_region_names)

print(regions_list)

['australiasoutheast', 'westus', 'india', 'centralus', 'asia', 'australia', 'unitedstateseuap', 'switzerland', 'westus2stage', 'northcentralus', 'eastusstage', 'francesouth', 'southeastasia', 'westcentralus', 'jioindiawest', 'eastus', 'southafrica', 'unitedstates', 'brazilus', 'jioindiacentral', 'canadaeast', 'norwaywest', 'northeurope', 'france', 'australiacentral2', 'brazilsoutheast', 'eastus2', 'westus2', 'northcentralusstage', 'southcentralusstg', 'singapore', 'europe', 'westindia', 'uaenorth', 'qatar', 'polandcentral', 'ukwest', 'switzerlandwest', 'uk', 'canadacentral', 'eastus2stage', 'southcentralusstage', 'global', 'centralusstage', 'australiacentral', 'sweden', 'southeastasiastage', 'switzerlandnorth', 'centraluseuap', 'southindia', 'poland', 'uksouth', 'germanynorth', 'japan', 'westus3', 'southcentralus', 'germanywestcentral', 'southafricanorth', 'israel', 'canada', 'qatarcentral', 'uaecentral', 'eastusstg', 'australiaeast', 'spaincentral', 'francecentral', 'norwayeast', 'eas

In [2]:
regions_successful = []
regions_failed = []

for region in regions_list:
    models_request_url = f"https://management.azure.com/subscriptions/{subscriptionId}/providers/Microsoft.CognitiveServices/locations/{region}/models?api-version=2023-05-01"
    models_response = requests.get(models_request_url, headers=headers)

    if models_response.status_code == 200:
        regions_successful.append(region)
    else:
        regions_failed.append(region)

print("Potential Azure OpenAI regions based on control plane response:")
print(regions_successful)

print("Azure OpenAI Not Supported:")
print(regions_failed)

Potential Azure OpenAI regions based on control plane response:
['westus', 'centralus', 'northcentralus', 'southeastasia', 'westcentralus', 'jioindiawest', 'eastus', 'jioindiacentral', 'canadaeast', 'northeurope', 'eastus2', 'westus2', 'uaenorth', 'polandcentral', 'ukwest', 'switzerlandwest', 'canadacentral', 'global', 'switzerlandnorth', 'southindia', 'uksouth', 'westus3', 'southcentralus', 'germanywestcentral', 'southafricanorth', 'qatarcentral', 'australiaeast', 'spaincentral', 'francecentral', 'norwayeast', 'koreacentral', 'westeurope', 'japanwest', 'japaneast', 'italynorth', 'centralindia', 'swedencentral', 'eastasia', 'brazilsouth']
Azure OpenAI Not Supported:
['australiasoutheast', 'india', 'asia', 'australia', 'unitedstateseuap', 'switzerland', 'westus2stage', 'eastusstage', 'francesouth', 'southafrica', 'unitedstates', 'brazilus', 'norwaywest', 'france', 'australiacentral2', 'brazilsoutheast', 'northcentralusstage', 'southcentralusstg', 'singapore', 'europe', 'westindia', 'qat

In [5]:
region_model_data = {}  

excluded_models =  ['text-similarity-ada-001', 'text-babbage-001', 'text-curie-001', 'text-similarity-curie-001', 'text-davinci-002','text-davinci-003', 'text-davinci-fine-tune-002', 'code-davinci-002', 'code-davinci-fine-tune-002','text-ada-001', 'text-search-ada-doc-001', 'text-search-ada-query-001', 'code-search-ada-code-001','code-search-ada-text-001', 'text-similarity-babbage-001', 'text-search-babbage-doc-001','text-search-babbage-query-001', 'code-search-babbage-code-001', 'code-search-babbage-text-001', 'text-search-curie-doc-001', 'text-search-curie-query-001', 'text-davinci-001','text-similarity-davinci-001', 'text-search-davinci-doc-001', 'text-search-davinci-query-001','code-cushman-001']

for region in regions_successful:
    model_request_url = f"https://management.azure.com/subscriptions/{subscriptionId}/providers/Microsoft.CognitiveServices/locations/{region}/models?api-version=2023-05-01"
    model_response = requests.get(model_request_url, headers=headers)

    response_dict = json.loads(model_response.text)
    #print(response_dict)
    data_test = []

    for item in response_dict["value"]:
        model_name = None
        version = None
        sku_name = None
        if item["model"]["capabilities"].get("scaleType") == "Manual": #skip legacy models
            continue
        model_name = item["model"]["name"]
        if model_name in excluded_models: # if in list skip
            continue
        version = item["model"]["version"]
        rdate = item["model"]["deprecation"]
        for sku in item["model"]["skus"]:
            sku_name = sku["name"]
        if sku_name == "Standard": # This example is only targeting Standard Model deployments SKUI
            data_test.append({"Model Name": model_name, "Version": version, "SKU Name": sku_name})
                #print(data_test)

    region_model_data[region] = data_test  # store the model data under corresponding region name

# Print result
for region, model_data in region_model_data.items():
    print(f'{region}: {model_data}')


            

westus: [{'Model Name': 'gpt-4', 'Version': 'vision-preview', 'SKU Name': 'Standard'}, {'Model Name': 'text-embedding-ada-002', 'Version': '2', 'SKU Name': 'Standard'}, {'Model Name': 'text-embedding-3-small', 'Version': '1', 'SKU Name': 'Standard'}, {'Model Name': 'gpt-4', 'Version': 'vision-preview', 'SKU Name': 'Standard'}, {'Model Name': 'text-embedding-ada-002', 'Version': '2', 'SKU Name': 'Standard'}, {'Model Name': 'text-embedding-3-small', 'Version': '1', 'SKU Name': 'Standard'}]
centralus: []
northcentralus: [{'Model Name': 'tts', 'Version': '001', 'SKU Name': 'Standard'}, {'Model Name': 'tts-hd', 'Version': '001', 'SKU Name': 'Standard'}, {'Model Name': 'whisper', 'Version': '001', 'SKU Name': 'Standard'}, {'Model Name': 'gpt-35-turbo-16k', 'Version': '0613', 'SKU Name': 'Standard'}, {'Model Name': 'text-embedding-ada-002', 'Version': '2', 'SKU Name': 'Standard'}, {'Model Name': 'babbage-002', 'Version': '1', 'SKU Name': 'Standard'}, {'Model Name': 'davinci-002', 'Version': '

In [6]:
rows = []
for region, models in region_model_data.items():
    for model in models:
        row = model.copy()  
        row['Region'] = region  
        rows.append(row)

df = pd.DataFrame(rows)
df = df[['Region', 'Model Name', 'Version', 'SKU Name']]
pd.set_option('display.max_rows', None)

df['Exist'] = True 
pivot_df = df.pivot_table(index='Region', columns=['Model Name', 'Version'], values='Exist', fill_value=False, aggfunc='any')
pivot_df.reset_index(inplace = True)

pivot_df

Model Name            Region babbage-002 dall-e-2 dall-e-3 davinci-002  \
Version                                1      2.0      3.0           1   
0              australiaeast       False    False     True       False   
1                brazilsouth       False    False    False       False   
2                 canadaeast       False    False    False       False   
3                     eastus       False     True     True       False   
4                    eastus2       False    False    False       False   
5              francecentral       False    False    False       False   
6                  japaneast       False    False    False       False   
7             northcentralus        True    False    False        True   
8                 norwayeast       False    False    False       False   
9              polandcentral       False    False    False       False   
10          southafricanorth       False    False    False       False   
11            southcentralus       False    False    False       False   
12                southindia       False    False    False       False   
13             swedencentral        True    False     True        True   
14          switzerlandnorth       False    False    False       False   
15                  uaenorth       False    False    False       False   
16                   uksouth       False    False    False       False   
17                westeurope       False    False    False       False   
18                    westus       False    False    False       False   
19                   westus3       False    False    False       False   

Model Name gpt-35-turbo               gpt-35-turbo-16k gpt-35-turbo-instruct  \
Version            0301   0613   1106             0613                  0914   
0                 False  False  False             True                 False   
1                 False  False  False            False                 False   
2                 False  False  False             True                 False   
3                  True  False  False             True                  True   
4                 False  False  False             True                 False   
5                  True   True   True             True                 False   
6                 False   True  False             True                 False   
7                 False  False  False             True                 False   
8                 False  False  False            False                 False   
9                 False  False  False            False                 False   
10                False  False  False            False                 False   
11                 True  False  False            False                 False   
12                False  False  False            False                 False   
13                False  False  False             True                  True   
14                False  False  False             True                 False   
15                False  False  False            False                 False   
16                 True  False  False             True                 False   
17                 True  False  False            False                 False   
18                False  False  False            False                 False   
19                False  False  False            False                 False   

Model Name        gpt-4                gpt-4-32k text-embedding-3-large  \
Version    1106-Preview vision-preview      0613                      1   
0                 False           True     False                   True   
1                 False          False     False                  False   
2                 False          False      True                   True   
3                 False          False     False                   True   
4                 False          False     False                   True   
5                 False          False     False                   True   
6         

In [15]:
region_model_quota_data = {}

# List of models to exclude
exclude_models = ["Code-Cushman-001", "code-cushman-fine-tune-002", "Code-Search-Ada-Code-001", "Code-Search-Ada-Text-001", "Text-Ada-001", "Text-Search-Ada-Doc-001", "Text-Search-Ada-Query-001", "Text-Similarity-Ada-001", "Babbage", "Code-Search-Babbage-Code-001", "Code-Search-Babbage-Text-001", "Text-Babbage-001", "Text-Search-Babbage-Doc-001", "Text-Search-Babbage-Query-001", "Text-Similarity-Babbage-001", "Curie", "Text-Curie-001", "Text-Search-Curie-Doc-001", "Text-Search-Curie-Query-001", "Text-Similarity-Curie-001", "Code-Davinci-002", "Code-Davinci-Fine-Tune-002", "Davinci", "Text-Davinci-001", "Text-Davinci-002", "Text-Davinci-003", "Text-Davinci-Fine-Tune-002", "Text-Search-Davinci-Doc-001", "Text-Search-Davinci-Query-001", "Text-Similarity-Davinci-001", "Code-Cushman-001", "code-cushman-fine-tune-002",  "Ada", "Code-Search-Ada-Code-001", "Code-Search-Ada-Text-001", "Text-Ada-001", "Text-Search-Ada-Doc-001", "Text-Search-Ada-Query-001", "Text-Similarity-Ada-001", "Babbage", "Code-Search-Babbage-Code-001", "Code-Search-Babbage-Text-001", "Text-Babbage-001", "Text-Search-Babbage-Doc-001", "Text-Search-Babbage-Query-001", "Text-Similarity-Babbage-001", "Curie", "Text-Curie-001", "Text-Search-Curie-Doc-001", "Text-Search-Curie-Query-001", "Text-Similarity-Curie-001", "Code-Davinci-Fine-Tune-002", "Davinci", "Text-Davinci-001", "Text-Davinci-002", "Text-Davinci-003", "Text-Davinci-Fine-Tune-002", "Text-Search-Davinci-Doc-001", "Text-Search-Davinci-Query-001", "Text-Similarity-Davinci-001"]

for region in regions_successful:
    url = f"https://management.azure.com/subscriptions/{subscriptionId}/providers/Microsoft.CognitiveServices/locations/{region}/usages?api-version=2023-05-01"
    response = requests.get(url, headers=headers)
    response_dict = json.loads(response.text)

    token_limit_details = []

    
    for item in response_dict['value']:
        # Ignoring 'OpenAI.LowPriority.gpt-4' value
        if item['name']['value'] == 'OpenAI.LowPriority.gpt-4':
            continue

        # Check if 'Tokens Per Minute' is in the localized value
        if 'Tokens Per Minute' in item['name']['localizedValue']:
        # Extract the localized value & limit.
            localized_value = item['name']['localizedValue']
            localized_value = localized_value.replace("Tokens Per Minute (thousands) - ", "")
            limit = item['limit']
            #print(localized_value)
            #print(limit)

            # If the model is not in the exclude list, add it to the token_limit_details
            if localized_value not in exclude_models:
                token_limit_details.append((localized_value, limit))
            
        elif 'Enqueued tokens' in item['name']['localizedValue']:
        # Extract the localized value & limit.
            localized_value = item['name']['localizedValue']
            #print(localized_value)
            localized_value = localized_value.replace("Enqueued tokens (thousands) - ", "")
            localized_value = localized_value + " - Global-Batch"
            #print(localized_value)
            limit = item['limit']
            #print(limit)
        
        # If the model is not in the exclude list, add it to the token_limit_details
            if localized_value not in exclude_models:
                token_limit_details.append((localized_value, limit))
            
    region_model_quota_data[region] = token_limit_details

print(region_model_quota_data)

{'westus': [('GPT-35-Turbo', 300), ('gpt-35-turbo - Global-Batch', 100000), ('GPT-4-Turbo', 80), ('GPT-4-Turbo - GlobalStandard', 450), ('GPT-4-Turbo-V', 30), ('GPT-4 - Global-Batch', 5000), ('GPT-4-Turbo - Global-Batch', 40000), ('GPT-4o - Global-Batch', 50000), ('GPT-4o-mini - Global-Batch', 50000), ('gpt-4o', 150), ('gpt-4o-mini', 450), ('gpt-4o - GlobalStandard', 450), ('gpt-4o-mini - GlobalStandard', 2000), ('gpt-4o - DataZoneStandard', 300), ('gpt-4o-mini - DataZoneStandard', 1000), ('o1', 50), ('o1 - GlobalStandard', 500), ('o1-mini', 50), ('o1-mini - GlobalStandard', 500), ('Text-Embedding-Ada-002', 350), ('text-embedding-3-small', 350)], 'centralus': [], 'northcentralus': [('GPT-35-Turbo', 300), ('GPT-35-Turbo - finetune', 250), ('GPT-35-Turbo-1106 - finetune', 250), ('GPT-35-Turbo-0125 - finetune', 250), ('gpt-35-turbo - Global-Batch', 100000), ('GPT-4-Turbo', 80), ('GPT-4-Turbo - GlobalStandard', 450), ('GPT-4 - finetune', 100), ('GPT-4o - finetune', 250), ('GPT-4o-mini - fi

In [23]:
flattened_data = [(region, lv, limit) for region in region_model_quota_data for lv, limit in region_model_quota_data[region]]

    ## Convert the list to a DataFrame
df = pd.DataFrame(flattened_data, columns=['Region', 'LocalizedValue', 'Limit'])
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 100)
 
    #print(df) 
pivot_df = df.pivot(index='Region', columns='LocalizedValue', values='Limit')
pivot_df.reset_index(level=0, inplace=True)


    # Specify the base column ordering you want
column_order = ["Region", "o1-mini", "o1", "gpt-4o", "gpt-4o-mini", "GPT-4", "GPT-4-32K", "GPT-4-Turbo", "GPT-4-Turbo-V", "GPT-35-Turbo", "GPT-35-Turbo-Instruct", "o1-mini - GlobalStandard",  "o1 - GlobalStandard", "gpt-4o - GlobalStandard", "gpt-4o-mini - GlobalStandard", "GPT-4-Turbo - GlobalStandard", "GPT-4o - Global-Batch", "GPT-4o-mini - Global-Batch", "GPT-4 - Global-Batch", "GPT-4-Turbo - Global-Batch", "gpt-35-turbo - Global-Batch", "Text-Embedding-Ada-002", "text-embedding-3-small", "text-embedding-3-large", "GPT-4o - finetune", "GPT-4o-mini - finetune", "GPT-4 - finetune",  "Babbage-002", "Babbage-002 - finetune","Davinci-002", "Davinci-002 - finetune","GPT-35-Turbo - finetune", "GPT-35-Turbo-1106 - finetune"]

# Create a set of column names from column_order and the current DataFrame
set_column_order = set(column_order)
set_existing_columns = set(pivot_df.columns)

# New columns which are not in the column_order list
new_columns = list(set_existing_columns - set_column_order)

# Add the new columns to the end of the column_order list
column_order = column_order + new_columns

# Reorder the dataframe
pivot_df = pivot_df[column_order]

pivot_df.columns.name = None
pivot_df.fillna(-1, inplace=True)
numeric_columns = pivot_df.columns.drop('Region') #Get all numeric columns

pivot_df[numeric_columns] = pivot_df[numeric_columns].astype(int)
pivot_df[numeric_columns] = pivot_df[numeric_columns].applymap(lambda x: str(x) if x != -1 else '-')



pivot_df

C:\Users\mrbul\AppData\Local\Temp\ipykernel_23052\3791504513.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_df[numeric_columns] = pivot_df[numeric_columns].applymap(lambda x: str(x) if x != -1 else '-')


,Region,o1-mini,o1,gpt-4o,gpt-4o-mini,GPT-4,GPT-4-32K,GPT-4-Turbo,GPT-4-Turbo-V,GPT-35-Turbo,GPT-35-Turbo-Instruct,o1-mini - GlobalStandard,o1 - GlobalStandard,gpt-4o - GlobalStandard,gpt-4o-mini - GlobalStandard,GPT-4-Turbo - GlobalStandard,GPT-4o - Global-Batch,GPT-4o-mini - Global-Batch,GPT-4 - Global-Batch,GPT-4-Turbo - Global-Batch,gpt-35-turbo - Global-Batch,Text-Embedding-Ada-002,text-embedding-3-small,text-embedding-3-large,GPT-4o - finetune,GPT-4o-mini - finetune,GPT-4 - finetune,Babbage-002,Babbage-002 - finetune,Davinci-002,Davinci-002 - finetune,GPT-35-Turbo - finetune,GPT-35-Turbo-1106 - finetune,gpt-4o-mini - DataZoneStandard,gpt-4o - DataZoneStandard,GPT-35-Turbo-0125 - finetune
0,australiaeast,-,-,-,-,40,80,80,30,300,-,-,-,450,2000,450,50000,50000,5000,40000,100000,350,350,350,-,-,-,-,-,-,-,-,-,-,-,-
1,brazilsouth,-,-,-,-,-,-,-,-,-,-,-,-,450,2000,450,-,-,-,-,-,350,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,canadaeast,-,-,-,-,40,80,80,-,300,-,-,-,450,2000,450,50000,50000,5000,40000,100000,350,350,350,-,-,-,-,-,-,-,-,-,-,-,-
3,eastus,50,50,150,450,-,-,80,-,240,240,500,500,450,2000,450,50000,50000,5000,40000,100000,240,350,350,-,-,-,-,-,-,-,-,-,1000,300,-
4,eastus2,50,50,150,450,-,-,80,-,300,-,500,500,450,2000,450,50000,50000,5000,40000,100000,350,350,350,250,-,-,-,-,-,-,250,250,1000,300,250
5,francecentral,-,-,-,-,20,60,80,-,240,-,-,-,450,2000,450,-,-,-,-,100000,240,-,350,-,-,-,-,-,-,-,-,-,1000,300,-
6,germanywestcentral,-,-,-,-,-,-,-,-,-,-,-,-,450,2000,450,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1000,300,-
7,japaneast,-,-,-,-,-,-,-,30,313,-,-,-,450,2000,450,-,-,-,-,-,350,350,350,-,-,-,-,-,-,-,-,-,-,-,-
8,koreacentral,-,-,-,-,-,-,-,-,-,-,-,-,450,2000,450,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
9,northcentralus,50,50,150,450,-,-,80,-,300,-,500,500,450,2000,450,50000,50000,5000,40000,100000,350,-,-,250,500,100,240,250,240,250,250,250,1000,300,250
